# **SCD type 2**

scd type2 on participant_attributes

In [0]:
%sql
--Create a table to implementation SCD Type 2 for Participant Attributes Silver Layer

CREATE OR REPLACE TABLE kusha_solutions.sai.participant_attributes_silver_scd2 (
  conversationId STRING,
  LegId STRING,
  NTLogin STRING,
  Leg_Ordinal INT,
  Transfer_leg INT,
  Alternate_Leg_Flag STRING,
  start_time TIMESTAMP,
  end_time TIMESTAMP,
  is_current BOOLEAN
)
USING DELTA;


In [0]:
%sql
--Initial load into SCD Type 2 table from Participant Attributes Silver Layer
INSERT INTO kusha_solutions.sai.participant_attributes_silver_scd2
SELECT 
  conversationId,
  LegId,
  NTLogin,
  Leg_Ordinal,
  Transfer_leg,
  Alternate_Leg_Flag,
  current_timestamp() AS start_time,
  NULL AS end_time,
  true AS is_current
FROM kusha_solutions.sai.participant_attributes_silver;


In [0]:
%sql
-- create a temp view for the updates
CREATE OR REPLACE TEMP VIEW participant_attributes_stage AS
SELECT * FROM VALUES
  ("0a57ee63-a99a-4476-995f-edbd9a3fb6c9", "0a57ee63-a99a-4476-995f-edbd9a3fb6c9-2", "GRANTAM", 2, 1, NULL),  -- same as before (no change)
  ("0a12fccd-06a0-4e52-8c64-98578139eea5", "0a12fccd-06a0-4e52-8c64-98578139eea5-1", "CXSMALINM1", 1, 1, NULL), -- Transfer_leg changed from 0 -> 1
  ("0a1ff49c-dc88-43f6-af60-0679583a2a77", "0a1ff49c-dc88-43f6-af60-0679583a2a77-1", "POWELLS2", 1, 0, NULL)   -- NTLogin changed
AS updates (
  conversationId, LegId, NTLogin, Leg_Ordinal, Transfer_leg, Alternate_Leg_Flag
);


In [0]:
%sql
--  Expire existing rows where data changed
MERGE INTO kusha_solutions.sai.participant_attributes_silver_scd2 AS tgt
USING participant_attributes_stage AS src
ON tgt.LegId = src.LegId AND tgt.is_current = true
WHEN MATCHED AND (
    tgt.NTLogin IS DISTINCT FROM src.NTLogin OR
    tgt.Leg_Ordinal IS DISTINCT FROM src.Leg_Ordinal OR
    tgt.Transfer_leg IS DISTINCT FROM src.Transfer_leg OR
    tgt.Alternate_Leg_Flag IS DISTINCT FROM src.Alternate_Leg_Flag
)
THEN UPDATE SET
  tgt.end_time = current_timestamp(),
  tgt.is_current = false;

--  Insert new rows for changes or new records
INSERT INTO kusha_solutions.sai.participant_attributes_silver_scd2
SELECT
  src.conversationId,
  src.LegId,
  src.NTLogin,
  src.Leg_Ordinal,
  src.Transfer_leg,
  src.Alternate_Leg_Flag,
  current_timestamp() AS start_time,
  NULL AS end_time,
  true AS is_current
FROM participant_attributes_stage AS src
LEFT JOIN kusha_solutions.sai.participant_attributes_silver_scd2 AS tgt
  ON src.LegId = tgt.LegId AND tgt.is_current = true
WHERE
  tgt.LegId IS NULL OR (
    tgt.NTLogin IS DISTINCT FROM src.NTLogin OR
    tgt.Leg_Ordinal IS DISTINCT FROM src.Leg_Ordinal OR
    tgt.Transfer_leg IS DISTINCT FROM src.Transfer_leg OR
    tgt.Alternate_Leg_Flag IS DISTINCT FROM src.Alternate_Leg_Flag
);


In [0]:
%sql
-- Check the results
SELECT * 
FROM kusha_solutions.sai.participant_attributes_silver_scd2
WHERE LegId IN (
  "0a57ee63-a99a-4476-995f-edbd9a3fb6c9-2",
  "0a12fccd-06a0-4e52-8c64-98578139eea5-1",
  "0a1ff49c-dc88-43f6-af60-0679583a2a77-1"
)
ORDER BY LegId, start_time;


scd type 2 on conversation_jobs

In [0]:
%sql
--Create a table to implementation SCD Type 2 for conversations Silver Layer
CREATE OR REPLACE TABLE kusha_solutions.sai.conversation_summary_scd2 (
  conversationId STRING,
  TimeZone STRING,
  disconnectType STRING,
  conference BOOLEAN,
  Talk_Time INT,
  Held_Time INT,
  ACW_Time INT,
  Handle_Time INT,
  Monitoring_Time INT,
  Voice_mail_Time INT,
  Transferred INT,
  Transferred_Blind INT,
  Transferred_Consult INT,
  Cobrowse INT,
  Consult INT,
  ACD_OB_Attempt INT,
  conversation_Date INT,
  Conversation_Start_Time TIMESTAMP,
  Conversation_End_Time TIMESTAMP,
  Conversation_Date_Key INT,
  Conversation_Start_Time_Key INT,
  Conversation_End_Time_Key INT,
  dnis STRING,
  Customer_Contact_Sequence INT,
  Previous_Contact_DateTime TIMESTAMP,
  Delta_Contact_DateTime STRING,
  destinationAddress STRING,
  -- SCD columns
  start_time TIMESTAMP,
  end_time TIMESTAMP,
  is_current BOOLEAN
)
USING DELTA;


In [0]:
%sql
--Initial load into SCD Type 2 table from conversation Silver Layer
INSERT INTO kusha_solutions.sai.conversation_summary_scd2
SELECT 
  conversationId,
  TimeZone,
  disconnectType,
  conference,
  Talk_Time,
  Held_Time,
  ACW_Time,
  Handle_Time,
  Monitoring_Time,
  Voice_mail_Time,
  Transferred,
  Transferred_Blind,
  Transferred_Consult,
  Cobrowse,
  Consult,
  ACD_OB_Attempt,
  conversation_Date,
  Conversation_Start_Time,
  Conversation_End_Time,
  Conversation_Date_Key,
  Conversation_Start_Time_Key,
  Conversation_End_Time_Key,
  dnis,
  Customer_Contact_Sequence,
  Previous_Contact_DateTime,
  Delta_Contact_DateTime,
  destinationAddress,
  current_timestamp() AS start_time,
  NULL AS end_time,
  TRUE AS is_current
FROM kusha_solutions.sai.conversation_jobs_silver;


In [0]:
%sql
-- create a temp view for testing
CREATE OR REPLACE TEMP VIEW conversation_stage AS
SELECT * FROM VALUES
  ("00d2cd83-293d-43b2-82c8-51e8354e97b6", "UK", NULL, false, 30, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 1, 20230921, 
   timestamp("2023-09-21T09:08:23.130"), timestamp("2023-09-21T09:08:42.770"), 20230921, 90823, 90842,
   "tel:+447552741002", 5, timestamp("2023-09-21T09:08:27.863"), "58081199", "tel:+447552741999"), -- changed Talk_Time, destinationAddress

  ("00ba27af-5804-4b79-87e7-da7543da8fda", "ES", NULL, false, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 20230921, 
   timestamp("2023-09-21T10:41:01.018"), timestamp("2023-09-21T10:41:07.097"), 20230921, 104101, 104107,
   "tel:+34603871075", 2, timestamp("2023-09-21T10:41:07.097"), "58075639", null) -- no change
AS updates (
  conversationId, TimeZone, disconnectType, conference, Talk_Time, Held_Time, ACW_Time, Handle_Time,
  Monitoring_Time, Voice_mail_Time, Transferred, Transferred_Blind, Transferred_Consult, Cobrowse,
  Consult, ACD_OB_Attempt, conversation_Date, Conversation_Start_Time, Conversation_End_Time,
  Conversation_Date_Key, Conversation_Start_Time_Key, Conversation_End_Time_Key,
  dnis, Customer_Contact_Sequence, Previous_Contact_DateTime, Delta_Contact_DateTime, destinationAddress
);


In [0]:
%sql
--  Expire changed records
MERGE INTO kusha_solutions.sai.conversation_summary_scd2 AS tgt
USING conversation_stage AS src
ON tgt.conversationId = src.conversationId AND tgt.is_current = true
WHEN MATCHED AND (
  tgt.Talk_Time IS DISTINCT FROM src.Talk_Time OR
  tgt.Handle_Time IS DISTINCT FROM src.Handle_Time OR
  tgt.destinationAddress IS DISTINCT FROM src.destinationAddress
)
THEN UPDATE SET
  tgt.end_time = current_timestamp(),
  tgt.is_current = false;

--  Insert new changed records
INSERT INTO kusha_solutions.sai.conversation_summary_scd2
SELECT 
  src.*, current_timestamp() AS start_time, NULL AS end_time, true AS is_current
FROM conversation_stage AS src
LEFT JOIN kusha_solutions.sai.conversation_summary_scd2 AS tgt
  ON src.conversationId = tgt.conversationId AND tgt.is_current = true
WHERE
  tgt.conversationId IS NULL OR (
    tgt.Talk_Time IS DISTINCT FROM src.Talk_Time OR
    tgt.Handle_Time IS DISTINCT FROM src.Handle_Time OR
    tgt.destinationAddress IS DISTINCT FROM src.destinationAddress
);


In [0]:
%sql
-- check the result
SELECT * 
FROM kusha_solutions.sai.conversation_summary_scd2
WHERE conversationId = '00d2cd83-293d-43b2-82c8-51e8354e97b6'
ORDER BY start_time;


test